In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import r2_score, mean_squared_error, root_mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from datetime import datetime, timedelta

In [2]:
light_on = pd.read_csv('../data/MergedData.csv')
light_on

/var/folders/nl/hqcw9pqd0ns4q0b61syv0q2m0000gn/T/ipykernel_28379/1966479485.py:1: DtypeWarning: Columns (16,22,23,26,27,29,30,31,32,33,34,35,36,38,42,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  light_on = pd.read_csv('../data/MergedData.csv')


,Unnamed: 0,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,...,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
1,1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,...,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057044,1187330,1248454,123904424,2020-03-06 14:00:26.000,Low (Severity Low) Catalyst Tank Level,NaN,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,...,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
1057045,1187331,1248455,123905139,2020-03-06 14:04:23.000,Condition Exists Engine Protection Torque Derate,NaN,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,...,32.0,True,98.6,18431,False,NaN,65.01096,NaN,73.2,7.83
1057046,1187332,1248456,123905996,2020-03-06 14:13:38.000,Abnormal Rate of Change Aftertreatment 1 Intak...,NaN,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,...,NaN,True,91.4,17407,NaN,NaN,66.5741,NaN,100.0,6.96
1057047,1187333,1248457,123906113,2020-03-06 14:14:13.000,Low (Severity Medium) Engine Coolant Level,NaN,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,...,NaN,True,100.4,1023,False,NaN,11.84489,14.1,100.0,1.74


cols = ['spn', 'fmi', 'AcceleratorPedal', 'BarometricPressure', 'CruiseControlSetSpeed', 'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'FuelTemperature', 'IntakeManifoldTemperature', 'LampStatus', 'Speed', 'Throttle', 'TurboBoostPressure']
light_on[cols] = light_on[cols].apply(pd.to_numeric, errors='coerce', axis=1)  #.reset_index(drop=True)
light_on

In [3]:
light_on['spn_fmi'] = light_on['spn'].astype(str) + '_' + light_on['fmi'].astype(str)
light_on

,Unnamed: 0,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,...,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,spn_fmi
0,0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,...,False,78.8,1023,True,NaN,0,3276.75,NaN,0,111_17
1,1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,...,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN,629_12
2,2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,...,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN,1807_2
3,3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,...,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN,1807_2
4,4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,...,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN,4364_17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057044,1187330,1248454,123904424,2020-03-06 14:00:26.000,Low (Severity Low) Catalyst Tank Level,NaN,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,...,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN,1761_17
1057045,1187331,1248455,123905139,2020-03-06 14:04:23.000,Condition Exists Engine Protection Torque Derate,NaN,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,...,True,98.6,18431,False,NaN,65.01096,NaN,73.2,7.83,1569_31
1057046,1187332,1248456,123905996,2020-03-06 14:13:38.000,Abnormal Rate of Change Aftertreatment 1 Intak...,NaN,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,...,True,91.4,17407,NaN,NaN,66.5741,NaN,100.0,6.96,3216_10
1057047,1187333,1248457,123906113,2020-03-06 14:14:13.000,Low (Severity Medium) Engine Coolant Level,NaN,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,...,True,100.4,1023,False,NaN,11.84489,14.1,100.0,1.74,111_18


In [4]:
dummy_columns = pd.get_dummies(light_on, columns = ['spn_fmi'])

In [5]:
#dummy_columns.loc[:, dummy_columns.columns.str.startswith('spn')] = dummy_columns.loc[:, dummy_columns.columns.str.startswith('spn')].astype('int')

In [6]:
#dummy_columns.to_csv('../data/TwentyRows_Preceding_SPN5246_LightOn_Dummy_Columns.csv')

In [7]:
dummy_columns

,Unnamed: 0,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,...,spn_fmi_96_3,spn_fmi_96_4,spn_fmi_96_9,spn_fmi_976_9,spn_fmi_97_15,spn_fmi_97_16,spn_fmi_97_3,spn_fmi_97_4,spn_fmi_98_18,spn_fmi_98_5
0,0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,...,False,False,False,False,False,False,False,False,False,False
1,1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,...,False,False,False,False,False,False,False,False,False,False
2,2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,...,False,False,False,False,False,False,False,False,False,False
3,3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,...,False,False,False,False,False,False,False,False,False,False
4,4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057044,1187330,1248454,123904424,2020-03-06 14:00:26.000,Low (Severity Low) Catalyst Tank Level,NaN,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,...,False,False,False,False,False,False,False,False,False,False
1057045,1187331,1248455,123905139,2020-03-06 14:04:23.000,Condition Exists Engine Protection Torque Derate,NaN,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,...,False,False,False,False,False,False,False,False,False,False
1057046,1187332,1248456,123905996,2020-03-06 14:13:38.000,Abnormal Rate of Change Aftertreatment 1 Intak...,NaN,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,...,False,False,False,False,False,False,False,False,False,False
1057047,1187333,1248457,123906113,2020-03-06 14:14:13.000,Low (Severity Medium) Engine Coolant Level,NaN,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False


In [8]:
dummy_columns['is_fullderate'] = (dummy_columns['spn'] == 5246).astype(int)

In [9]:
dummy_columns.sort_values(['EquipmentID', 'EventTimeStamp'])
dummy_columns['is_fullderate_group'] = dummy_columns.groupby('EquipmentID')['is_fullderate'].cumsum()

In [10]:
Num_of_FullDerates_inDataSet = dummy_columns['is_fullderate_group'].sum()
Num_of_FullDerates_inDataSet # 1

np.int64(394678)

In [11]:
dummy_columns['EquipID_Index'] = dummy_columns['EquipmentID'].astype(str) + '_' + dummy_columns['is_fullderate_group'].astype(str)

dummy_columns.to_csv('../data/TwentyRows_Preceding_SPN5246_LightOn_Truck_Identifier_SubsequentTrialRun.csv')

In [12]:
def check_time_intervals_by_group(df, group_column, timestamp_column, condition_column=None, 
                                 condition_value=None, interval_column_name='time_to_next_condition'):
    """
    Calculate the time interval between the current row and the next row(s) that meet a specific condition,
    grouped by a specific column (e.g., EquipmentID for trucks).
    
    Parameters:
    -----------
    df : pandas.DataFrame
        The dataframe containing the timestamp, condition, and group columns.
    group_column : str
        The name of the column to group by (e.g., 'EquipmentID').
    timestamp_column : str
        The name of the column containing datetime values.
    condition_column : str, optional
        The name of the column to check for the condition. If None, all rows are considered.
    condition_value : any, optional
        The value to check for in the condition column. If None, all rows are considered.
    interval_column_name : str, optional
        The name for the new column that will store the time interval information.
        Default is 'time_to_next_condition'.
    
    Returns:
    --------
    pandas.DataFrame
        The original dataframe with an additional column showing the time interval
        to the next row meeting the condition within each group.
    """
    # Ensure the timestamp column is in datetime format
    df[timestamp_column] = pd.to_datetime(df[timestamp_column])
    
    # Create a copy of the dataframe to avoid modifying the original
    result_df = df.copy()
    
    # Add a column to store the time intervals with the specified name
    result_df[interval_column_name] = pd.NaT
    
    # Get unique group values (e.g., truck IDs)
    groups = df[group_column].unique()
    
    # Process each group (e.g., each truck) separately
    for group_value in groups:
        # Get data for just this group
        group_mask = df[group_column] == group_value
        group_df = df[group_mask]
        
        # Sort the group by timestamp to ensure proper ordering
        group_df = group_df.sort_values(by=timestamp_column)
        
        # Filter rows that meet the condition if a condition is specified
        if condition_column is not None and condition_value is not None:
            condition_rows = group_df[group_df[condition_column] == condition_value]
        else:
            condition_rows = group_df
        
        # If this group doesn't have any rows meeting the condition, continue to next group
        if len(condition_rows) == 0:
            continue
            
        # For each row in this group
        for idx, row in group_df.iterrows():
            current_time = row[timestamp_column]
            
            # Find the next row that meets the condition and is after the current time
            next_condition_rows = condition_rows[condition_rows[timestamp_column] > current_time]
            
            if not next_condition_rows.empty:
                # Get the earliest next condition time
                next_condition_time = next_condition_rows[timestamp_column].min()
                
                # Calculate the time difference
                time_diff = next_condition_time - current_time
                
                # Store the result in the specified column for this row
                result_df.loc[idx, interval_column_name] = time_diff
    
    return result_df

In [13]:
TimeIntervalColumn_AddedTo_DummyColumns = check_time_intervals_by_group(dummy_columns, group_column='EquipmentID', timestamp_column='EventTimeStamp', condition_column='spn', condition_value=5246, interval_column_name='time_to_next_SPN5246') 
TimeIntervalColumn_AddedTo_DummyColumns

/var/folders/nl/hqcw9pqd0ns4q0b61syv0q2m0000gn/T/ipykernel_28379/2174836225.py:75: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1220 days 02:29:40' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  result_df.loc[idx, interval_column_name] = time_diff


,Unnamed: 0,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,...,spn_fmi_97_15,spn_fmi_97_16,spn_fmi_97_3,spn_fmi_97_4,spn_fmi_98_18,spn_fmi_98_5,is_fullderate,is_fullderate_group,EquipID_Index,time_to_next_SPN5246
0,0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,...,False,False,False,False,False,False,0,0,1439_0,NaT
1,1,2,990360,2015-02-21 11:34:34,NaN,NaN,unknown,unknown,unknown,unknown,...,False,False,False,False,False,False,0,0,1439_0,NaT
2,2,3,990364,2015-02-21 11:35:31,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,...,False,False,False,False,False,False,0,0,1369_0,NaT
3,3,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,...,False,False,False,False,False,False,0,0,1369_0,NaT
4,4,5,990416,2015-02-21 11:39:41,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,...,False,False,False,False,False,False,0,0,1674_0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057044,1187330,1248454,123904424,2020-03-06 14:00:26,Low (Severity Low) Catalyst Tank Level,NaN,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,...,False,False,False,False,False,False,0,0,2282_0,NaT
1057045,1187331,1248455,123905139,2020-03-06 14:04:23,Condition Exists Engine Protection Torque Derate,NaN,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,...,False,False,False,False,False,False,0,0,1994_0,NaT
1057046,1187332,1248456,123905996,2020-03-06 14:13:38,Abnormal Rate of Change Aftertreatment 1 Intak...,NaN,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,...,False,False,False,False,False,False,0,0,1850_0,NaT
1057047,1187333,1248457,123906113,2020-03-06 14:14:13,Low (Severity Medium) Engine Coolant Level,NaN,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,...,False,False,False,False,False,False,0,0,2377_0,NaT


In [14]:
def add_interval_classification(df, interval_column='time_to_next_SPN5246', classification_column='interval_class'):
    """
    Adds a classification column based on time interval thresholds:
    - 0 if interval > 6 hours
    - 1 if interval is between 2 and 6 hours (inclusive)
    - 2 if interval < 2 hours
    
    Parameters:
    -----------
    df : pandas.DataFrame
        The dataframe containing the time interval column.
    interval_column : str
        The name of the column containing time interval values (timedelta format).
    classification_column : str, optional
        The name for the new classification column.
        Default is 'interval_class'.
    
    Returns:
    --------
    pandas.DataFrame
        The dataframe with the added classification column.
    """
    # Create a copy of the dataframe to avoid modifying the original
    result_df = df.copy()
    
    # Initialize the classification column with default value 0
    result_df[classification_column] = 0
    
    # Apply classification rules
    # First convert timedeltas to hours for easier comparison
    hours_series = pd.Series([td.total_seconds()/3600 if not pd.isna(td) else None 
                             for td in result_df[interval_column]])
    
    # Apply the classification rules:
    # 1 if between 2 and 6 hours inclusive
    result_df.loc[(hours_series <= 6) & (hours_series >= 2), classification_column] = 1
    
    # 2 if less than 2 hours
    result_df.loc[hours_series < 2, classification_column] = 2
    
    return result_df


In [15]:
TimeIntervalClass_AddedTo_DummyColumns = add_interval_classification(TimeIntervalColumn_AddedTo_DummyColumns, 'time_to_next_SPN5246', classification_column='time_interval_to_SPN5246_class')
TimeIntervalClass_AddedTo_DummyColumns

,Unnamed: 0,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,...,spn_fmi_97_16,spn_fmi_97_3,spn_fmi_97_4,spn_fmi_98_18,spn_fmi_98_5,is_fullderate,is_fullderate_group,EquipID_Index,time_to_next_SPN5246,time_interval_to_SPN5246_class
0,0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,...,False,False,False,False,False,0,0,1439_0,NaT,0
1,1,2,990360,2015-02-21 11:34:34,NaN,NaN,unknown,unknown,unknown,unknown,...,False,False,False,False,False,0,0,1439_0,NaT,0
2,2,3,990364,2015-02-21 11:35:31,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,...,False,False,False,False,False,0,0,1369_0,NaT,0
3,3,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,...,False,False,False,False,False,0,0,1369_0,NaT,0
4,4,5,990416,2015-02-21 11:39:41,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,...,False,False,False,False,False,0,0,1674_0,NaT,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057044,1187330,1248454,123904424,2020-03-06 14:00:26,Low (Severity Low) Catalyst Tank Level,NaN,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,...,False,False,False,False,False,0,0,2282_0,NaT,0
1057045,1187331,1248455,123905139,2020-03-06 14:04:23,Condition Exists Engine Protection Torque Derate,NaN,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,...,False,False,False,False,False,0,0,1994_0,NaT,0
1057046,1187332,1248456,123905996,2020-03-06 14:13:38,Abnormal Rate of Change Aftertreatment 1 Intak...,NaN,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,...,False,False,False,False,False,0,0,1850_0,NaT,0
1057047,1187333,1248457,123906113,2020-03-06 14:14:13,Low (Severity Medium) Engine Coolant Level,NaN,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,...,False,False,False,False,False,0,0,2377_0,NaT,0


In [16]:
TimeIntervalClass_AddedTo_DummyColumns.to_csv('../data/MergedFullDataSet_TimeIntervalClass_AddedTo_DummyColumns.csv')

KeyboardInterrupt: 

potentially faster alternative approach: 

In [ ]:
def check_time_intervals_optimized(df, EventTimeStamp, condition_column='spn', condition_value=5246, interval_column_name='time_to_next_SPN5246'):
    """
    Optimized version for large datasets that calculates time intervals more efficiently.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        The dataframe containing the timestamp and condition columns.
    EventTimeStamp : str
        The name of the column containing datetime values.
    condition_column : str, optional
        The name of the column to check for the condition. If None, all rows are considered.
    condition_value : any, optional
        The value to check for in the condition column. If None, all rows are considered.
    interval_column_name : str, optional
        The name for the new column that will store the time interval information.
        Default is 'time_to_next_condition'.
    
    Returns:
    --------
    pandas.DataFrame
        The original dataframe with an additional column showing the time interval
        to the next row meeting the condition.
    """
    # Ensure timestamps are in datetime format
    df[EventTimeStamp] = pd.to_datetime(df[EventTimeStamp])
    
    # Create a copy of the dataframe
    result_df = df.copy()
    result_df[interval_column_name] = pd.NaT
    
    # If no condition is specified, calculate time to next row
    if condition_column is None or condition_value is None:
        # Sort by timestamp to ensure correct calculations
        df_sorted = df.sort_values(by=EventTimeStamp).reset_index(drop=False)
        
        for i in range(len(df_sorted) - 1):
            curr_idx = df_sorted.index[i]
            next_idx = df_sorted.index[i + 1]
            time_diff = df_sorted.iloc[i + 1][EventTimeStamp] - df_sorted.iloc[i][EventTimeStamp]
            result_df.loc[curr_idx, interval_column_name] = time_diff
            
    else:
        # Filter rows that meet the condition
        condition_mask = df[condition_column] == condition_value
        condition_indices = df.index[condition_mask].tolist()
        
        # For each row, find the next timestamp that meets the condition
        for idx, row in df.iterrows():
            current_time = row[EventTimeStamp]
            
            # Find the next time that meets the condition
            future_times = [t for t in df.loc[condition_indices, EventTimeStamp] if t > current_time]
            
            if future_times:
                next_time = min(future_times)
                result_df.loc[idx, interval_column_name] = next_time - current_time
    
    return result_df

In [ ]:
TimeIntervalColumn_AddedTo_DummyColumns_Ver2 = check_time_intervals_optimized(df, EventTimeStamp, condition_column='spn', condition_value=5246, interval_column_name='time_to_next_SPN5246')
TimeIntervalColumn_AddedTo_DummyColumns_Ver2

NameError: name 'df' is not defined

In [ ]:
TimeIntervalClass_AddedTo_DummyColumns_Ver2 = add_interval_classification(TimeIntervalColumn_AddedTo_DummyColumns_Ver2, 'time_to_next_SPN5246', classification_column='time_interval_to_SPN5246_class')
TimeIntervalClass_AddedTo_DummyColumns_Ver2

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,...,spn_fmi_96_9,spn_fmi_976_9,spn_fmi_97_15,spn_fmi_97_16,spn_fmi_98_18,is_fullderate,is_fullderate_group,EquipID_Index,time_to_next_SPN5246,time_interval_to_SPN5246_class
0,483365,9355560,2016-05-23 12:54:05,NaN,NaN,PC4__1284P4C_2*,6U13D13,MX,PCAR,0,...,False,False,False,False,False,0,0,301_0,0 days 03:04:19,1
1,483393,9356593,2016-05-23 13:36:29,NaN,NaN,PC4__1284P4C_2*,6U13D13,MX,PCAR,0,...,False,False,False,False,False,0,0,301_0,0 days 02:21:55,1
2,483447,9357988,2016-05-23 14:45:09,NaN,NaN,PC4__1284P4C_2*,6U13D13,MX,PCAR,0,...,False,False,False,False,False,0,0,301_0,0 days 01:13:15,2
3,483879,9368775,2016-05-24 06:39:21,NaN,NaN,PC4__1284P4C_2*,6U13D13,MX,PCAR,0,...,False,False,False,False,False,0,0,301_0,0 days 13:22:37,0
4,484033,9373134,2016-05-24 09:32:10,NaN,NaN,PC4__1284P4C_2*,6U13D13,MX,PCAR,0,...,False,False,False,False,False,0,0,301_0,0 days 10:29:48,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4671,1053427,60808479,2018-09-06 10:50:43,Condition Exists NOx limits exceeded due to In...,NaN,05317106*04075152*092613211021*09300006*G1*BDR*,79731935,6X1u13D1500000000,CMMNS,0,...,False,False,False,False,False,0,0,306_0,1 days 00:31:57,0
4672,1053620,60904453,2018-09-07 05:45:39,Data Drifted High Aftertreatment 1 Outlet NOx,NaN,05317106*04075152*092613211021*09300006*G1*BDR*,79731935,6X1u13D1500000000,CMMNS,0,...,False,False,False,False,False,0,0,306_0,0 days 05:37:01,1
4673,1053622,60904613,2018-09-07 05:48:43,Condition Exists NOx limits exceeded due to In...,NaN,05317106*04075152*092613211021*09300006*G1*BDR*,79731935,6X1u13D1500000000,CMMNS,0,...,False,False,False,False,False,0,0,306_0,0 days 05:33:57,1
4674,1053642,60909415,2018-09-07 06:47:05,Condition Exists Engine Protection Torque Derate,NaN,05317106*04075152*092613211021*09300006*G1*BDR*,79731935,6X1u13D1500000000,CMMNS,0,...,False,False,False,False,False,0,0,306_0,0 days 04:35:35,1


In [ ]:
TimeIntervalColumn_AddedTo_DummyColumns_Ver2.dtypes

RecordID                           int64
ESS_Id                             int64
EventTimeStamp            datetime64[ns]
eventDescription                  object
actionDescription                float64
                               ...      
spn_fmi_98_18                       bool
is_fullderate                      int64
is_fullderate_group                int64
EquipID_Index                     object
time_to_next_condition    datetime64[ns]
Length: 425, dtype: object